Il k-nearest neighbors è un algoritmo di classificazione basato sull'assunto che le osservazioni simili appartengano alla stessa classe. Consiste nel calcolare la distanza tra un'osservazione di test e tutte le osservazioni del dataset di training e nell'assegnare l'etichetta della classe più presente tra i k osservazioni più vicine.

Il processo è questo:

1) Scegliere il numero di vicini k da considerare durante il calcolo delle distanze

2) Calcolare la distanza tra l'osservazione di test e tutte le osservazioni del dataset di training

3) Selezionare le k osservazioni più vicine

4) Assegnare l'etichetta della classe più presente tra le k osservazioni più vicine all'osservazione di test

Questo algoritmo può essere lento nel caso di dataset di grandi dimensioni e l'accuratezza del modello può essere influenzata in modo significativo dalla scelta del numero di vicini k.


Pro: 

Semplicità: è un algoritmo semplice e facile da capire e implementare

Buona prestazione con dataset di piccole dimensioni

Contro:

Lentezza con dataset di grandi dimensioni perchè deve calcolare la distanza tra l'osservazione di test e tutte le osservazioni ogni volta. Infatti è considerato un algoritmo lazy learner, ossia un algoritmo che richiede meno tempo per l'apprendimento e più tempo per la classificazione

Sensibilità alla scelta del numero k di vicini

Sensibilità alla presenza di outliers

In [23]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")
df <- df_finale

In [26]:
names(df) <- c('Centre', 'Gender', 'Age', 'LHOSpretx', 'Status', 'VaricealBleed', 'Ascites', 'Tips',  
                'PVthrombosis', 'NutritStatus', 'Previoussurg', 'Ventilationpretx', 'Vasopressors', 
               'InfusionAlb', 'PaO2FiO2', 'PlatCount', 'Albumin', 'Duration', 'PRBCintraop', 'FFPintraop', 'Plateletsintraop',
               'Fibrinogen', 'CRIO', 'Crystalloids', 'PPS5', 'Norepinend', 'Lactates', 'ColdIschemia', 'ReperfusSy', 'PCshunt',
               'Cavaveinanasth', 'VVBypass', 'Biliaryanasth', 'Openabdomen', 'Donorgender', 'Donortype', 'Organtype' ,'DonorECMO', 
               'RegionalPerf', 'OrganBiopsy', 'Machineperf', 'DonorSite', 'DonorCauseDeath', 'DonorAge', 'DonorHeight', 'DonorSodiemia',
               'DonorGGT', 'MELDNa', 'Alcoholic', 'cirrhosisDrugs', 'NASH', 'Lateretx', 'DonorMaastricht', 'DonorSteatosis' ,
               'HypoNormoter', 'Nocomorbodities', 'Hypertension', 'SeverePsychosis', 'Periodstx', 'Nighttx', 'Admsite' ,
               'Encephalopathy', 'Cholestatic', 'Rare', 'Viral', 'HCC', 'Respcomorbidities', 'CVcomorbidities', 'Neurocomorbidities',
               'Renalcomorbidities', 'Diabetes', 'Anticoagulation', 'MELDExcept', 'deg_cat')

# Feature selection

Feature selection fatta attraverso l'algoritmo boruta che si basa sull'algoritmo di random forest e utilizza una tecnica di confronto dei risultati ottenuti con un insieme di dati di controllo per determinare quali variabili sono significative. è un algoritmo che tiene conto delle interazioni tra variabili, è versatile in quanto lavora sia su variabili continue che categoriche ed è resistente agli outliers

In [27]:
library(Boruta)
# Decide if a variable is important or not using Boruta
boruta_output <- Boruta(deg_cat ~ ., data=df, doTrace=2)  # perform Boruta search


boruta_signif <- names(boruta_output$finalDecision[boruta_output$finalDecision %in% c("Confirmed", "Tentative")])  # collect Confirmed and Tentative variables
print(boruta_signif)  # significant variables


plot(boruta_output, cex.axis=1.2, las=2, xlab="", main="Variable Importance")  # plot variable importance


df_boruta <- df[,boruta_signif]

df_boruta <- apply(select_if(df,is.matrix),as.numeric)


 1. run of importance source...

 2. run of importance source...

 3. run of importance source...

 4. run of importance source...

 5. run of importance source...

 6. run of importance source...

 7. run of importance source...

 8. run of importance source...

 9. run of importance source...

 10. run of importance source...

 11. run of importance source...

 12. run of importance source...

 13. run of importance source...

After 13 iterations, +1.3 mins: 

 confirmed 20 attributes: Admsite, Albumin, Ascites, Centre, CRIO and 15 more;

 rejected 30 attributes: Age, Alcoholic, Anticoagulation, Cavaveinanasth, Cholestatic and 25 more;

 still have 23 attributes left.


 14. run of importance source...

 15. run of importance source...

 16. run of importance source...

 17. run of importance source...

 18. run of importance source...

After 18 iterations, +1.7 mins: 

 rejected 5 attributes: DonorAge, Donortype, Periodstx, Renalcomorbidities, Ventilationpretx;

 still have 18 attri

 [1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "Fibrinogen"       "CRIO"             "Crystalloids"     "PPS5"            
[17] "Norepinend"       "Lactates"         "ReperfusSy"       "Biliaryanasth"   
[21] "Openabdomen"      "DonorSodiemia"    "DonorGGT"         "MELDNa"          
[25] "Nocomorbodities"  "Admsite"          "Encephalopathy"   "HCC"             


ERROR: Error in match.fun(FUN): l'argomento "FUN"  non è specificato e non ha un valore predefinito


In [28]:
par(mar = c(15, 5, 2, 0.1))  
plot(boruta_output, cex.axis=1.1, las=2, xlab="", main="Variable Importance")

In [29]:
df_boruta$deg_cat <- df$deg_cat

In [30]:
boruta_signif #variabili selezionate

[1] "Centre"           "LHOSpretx"        "Ascites"          "NutritStatus"    
 [5] "InfusionAlb"      "PaO2FiO2"         "PlatCount"        "Albumin"         
 [9] "Duration"         "PRBCintraop"      "FFPintraop"       "Plateletsintraop"
[13] "Fibrinogen"       "CRIO"             "Crystalloids"     "PPS5"            
[17] "Norepinend"       "Lactates"         "ReperfusSy"       "Biliaryanasth"   
[21] "Openabdomen"      "DonorSodiemia"    "DonorGGT"         "MELDNa"          
[25] "Nocomorbodities"  "Admsite"          "Encephalopathy"   "HCC"

## Data partition

Viene effettuata divisione tra training e test set, che è del 70-30 per tutti i modelli utilizzati

In [31]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)



Viene effettuato un bilanciamento della variabile target perchè in origine la classe ottimo conteneva circa 1/4 dei record rispetto alle altre due classi e i modelli facevano fatica a classificare quel determinato valore. La funzione utilizzata è SmoteClassif() che:

1) Seleziona a caso una osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

2) Seleziona a caso un'altra osservazione della classe minoritaria e ne calcola la distanza rispetto alle altre osservazioni della stessa classe

3) Genera una nuova osservazione sintetica che si trova a metà strada tra le due osservazioni selezionate in precedenza

4) Ripete i passaggi precedenti fino a raggiungere il bilanciamento desiderato


Questa funzione utilizza il knn per calcolare le distanze tra le osservazioni e generare le osservazioni, infatti il valore di k è stato selezionato pari alla radice quadrata del numero totale di record presenti nel training set come si fa per il knn normalmente per il knn



In [32]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    555     555 

Viene effettuato lo stesso processo considerando il dataset dopo la boruta feature selection invece del dataset con tutte le feature in modo da poter fare una comparazione sui risultati successivamente

In [33]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df_boruta$deg_cat, SplitRatio = 0.7)
trainB  <- subset(df_boruta, sample == TRUE)
testB   <- subset(df_boruta, sample == FALSE)



TestClassesB <- testB$deg_cat
TestDataB <- subset(testB,select= -deg_cat)


In [34]:
trainB <- SmoteClassif(deg_cat ~ ., trainB, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(trainB$deg_cat)

TrainClassesB <- trainB$deg_cat
TrainDataB <- subset(trainB,select= -deg_cat)


   fast NotFast 
    555     555 

## Models

Attraverso la funzione trainControl viene effettuata una repeated cross validation e una random search.

La prima permette di ottenere una stima più accurata dell'accuratezza del modello:

1) Dividere il dataset in un numero predeterminato di folds (in questo caso è stato scelto n=10 quindi 10 folds)

2) Eseguire la cross-validation ripetutamente utilizzando ogni fold come test set e gli altri folds come training set

3) Calcolare la media e la deviazione standard delle misure di accuratezza ottenute per ogni fold

4) Valutare l'accuratezza e la robustezza del modello utilizzando la media e la deviazione standard delle misure di accuratezza

L'utilizzo della repeated cross validation è particolarmente importante quando si lavora con dataset di dimensioni ridotte o con un numero limitato di osservazioni perchè in questi casi i risultati possono essere influenzata dall'aleatorietà nella divisione dei dati in training set e test set. La repeated cross-validation permette quindi di ottenere una stima più accurata dell'accuratezza del modello ripetendo il processo  più volte e considerando dati diversi



La random search, invece è una tecnica di ottimizzazione dei modelli che viene utilizzata per selezionare i migliori parametri di un modello. Consiste nel generare a caso diverse combinazioni di parametri e nel valutare l'accuratezza del modello per ogni combinazione. La combinazione di parametri che ottiene il risultato migliore viene quindi selezionata come quella ottimale. Questa soluzione è stata preferita alla grid search che poteva dare risultati migliori per una questione di costo computazionale

In [35]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

Viene allenato il modello knn sul dataset senza feature selection

In [36]:
model_knn <- train(x=TrainData, y=TrainClasses, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax=293, distance=1.1582, kernel=cos 
- Fold01.Rep1: kmax=293, distance=1.1582, kernel=cos 
+ Fold01.Rep1: kmax=  7, distance=2.9451, kernel=biweight 
- Fold01.Rep1: kmax=  7, distance=2.9451, kernel=biweight 
+ Fold01.Rep1: kmax= 43, distance=0.9388, kernel=triweight 
- Fold01.Rep1: kmax= 43, distance=0.9388, kernel=triweight 
+ Fold02.Rep1: kmax=293, distance=1.1582, kernel=cos 
- Fold02.Rep1: kmax=293, distance=1.1582, kernel=cos 
+ Fold02.Rep1: kmax=  7, distance=2.9451, kernel=biweight 
- Fold02.Rep1: kmax=  7, distance=2.9451, kernel=biweight 
+ Fold02.Rep1: kmax= 43, distance=0.9388, kernel=triweight 
- Fold02.Rep1: kmax= 43, distance=0.9388, kernel=triweight 
+ Fold03.Rep1: kmax=293, distance=1.1582, kernel=cos 
- Fold03.Rep1: kmax=293, distance=1.1582, kernel=cos 
+ Fold03.Rep1: kmax=  7, distance=2.9451, kernel=biweight 
- Fold03.Rep1: kmax=  7, distance=2.9451, kernel=biweight 
+ Fold03.Rep1: kmax= 43, distance=0.9388, kernel=triweight 
- Fold03.Rep1: kmax= 4

Viene allenato il modello knn sul dataset con feature selection

In [37]:
model_knnB <- train(x=TrainDataB, y=TrainClassesB, 
               method = "kknn",
               trControl = ctrl) 

+ Fold01.Rep1: kmax=277, distance=0.3293, kernel=gaussian 
- Fold01.Rep1: kmax=277, distance=0.3293, kernel=gaussian 
+ Fold01.Rep1: kmax=307, distance=1.2862, kernel=gaussian 
- Fold01.Rep1: kmax=307, distance=1.2862, kernel=gaussian 
+ Fold01.Rep1: kmax=159, distance=1.9323, kernel=epanechnikov 
- Fold01.Rep1: kmax=159, distance=1.9323, kernel=epanechnikov 
+ Fold02.Rep1: kmax=277, distance=0.3293, kernel=gaussian 
- Fold02.Rep1: kmax=277, distance=0.3293, kernel=gaussian 
+ Fold02.Rep1: kmax=307, distance=1.2862, kernel=gaussian 
- Fold02.Rep1: kmax=307, distance=1.2862, kernel=gaussian 
+ Fold02.Rep1: kmax=159, distance=1.9323, kernel=epanechnikov 
- Fold02.Rep1: kmax=159, distance=1.9323, kernel=epanechnikov 
+ Fold03.Rep1: kmax=277, distance=0.3293, kernel=gaussian 
- Fold03.Rep1: kmax=277, distance=0.3293, kernel=gaussian 
+ Fold03.Rep1: kmax=307, distance=1.2862, kernel=gaussian 
- Fold03.Rep1: kmax=307, distance=1.2862, kernel=gaussian 
+ Fold03.Rep1: kmax=159, distance=1.9323

I parametri testati sono: 

kmax = numero di nearest neighbors considerati

distance = parametro per la distanza minkowski

kernel = per specificare il metodo di con cui vengono calcolati i pesi da utilizzare durante il calcolo delle distanze tra le osservazioni

## Results

Vengono analizzati i risultati ottenuti dai due modelli durante la cross validation. Le metriche utilizzate sono: accuracy e kappa. La prima si riferisce alla percentuale di osservazioni correttamente classificate dal modello rispetto al totale di osservazioni. La seconda viene utilizzata per misurare l'accordo tra più modelli di classificazione. Può assumere valori compresi tra -1 e 1. Un valore di kappa vicino a 1 indica un alto livello di accordo tra le classificazioni, mentre un valore di kappa vicino a 0 indica un accordo simile a quello ottenuto per caso. Un valore di kappa negativo indica un accordo peggiore di quello ottenuto per caso.

In [38]:
print(model_knn)
plot(model_knn)

k-Nearest Neighbors 

1110 samples
  73 predictor
   2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 999, 998, 1000, 1000, 998, 1000, ... 
Resampling results across tuning parameters:

  kmax  distance   kernel     Accuracy   Kappa    
    7   2.9451422  biweight   0.6656766  0.3313102
   43   0.9388447  triweight  0.6990274  0.3980433
  293   1.1582469  cos        0.6891089  0.3781515

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 43, distance = 0.9388447
 and kernel = triweight.


In [39]:
print(model_knnB)
plot(model_knnB)

k-Nearest Neighbors 

1110 samples
  28 predictor
   2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 1 times) 
Summary of sample sizes: 1000, 999, 998, 999, 999, 999, ... 
Resampling results across tuning parameters:

  kmax  distance   kernel        Accuracy   Kappa    
  159   1.9323371  epanechnikov  0.7548824  0.5097831
  277   0.3293081  gaussian      0.7243153  0.4486413
  307   1.2861947  gaussian      0.7458814  0.4918319

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were kmax = 159, distance = 1.932337
 and kernel = epanechnikov.


Vengono messi a confronto i risultati ottenuti sulla cross validation dei due modelli in modo da selezionare il più performante da utilizzare nel test set

In [40]:
# collect resamples
results <- resamples(list(knn=model_knn,knn_boruta=model_knnB))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: knn, knn_boruta 
Number of resamples: 10 

Accuracy 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
knn        0.6428571 0.6643939 0.7027027 0.6990274 0.7262581 0.7589286    0
knn_boruta 0.6756757 0.7133292 0.7612613 0.7548824 0.7810197 0.8660714    0

Kappa 
                Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
knn        0.2857143 0.3293944 0.4034202 0.3980433 0.4525162 0.5178571    0
knn_boruta 0.3506662 0.4273243 0.5223882 0.5097831 0.5616968 0.7321429    0


## Prediction

Viene selezionato il modello più performante per usarlo nel test set ed avere risultati definitivi.

In [41]:
pred_knn <- predict(model_knnB,test)  
pred_knn.prob <- predict(model_knnB,test, type="prob")


result <- confusionMatrix(TestClasses,  pred_knn) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     127      44
   NotFast   94     211
                                          
               Accuracy : 0.7101          
                 95% CI : (0.6671, 0.7505)
    No Information Rate : 0.5357          
    P-Value [Acc > NIR] : 5.454e-15       
                                          
                  Kappa : 0.4083          
                                          
 Mcnemar's Test P-Value : 3.031e-05       
                                          
            Sensitivity : 0.5747          
            Specificity : 0.8275          
         Pos Pred Value : 0.7427          
         Neg Pred Value : 0.6918          
             Prevalence : 0.4643          
         Detection Rate : 0.2668          
   Detection Prevalence : 0.3592          
      Balanced Accuracy : 0.7011          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5746606            0.8274510            0.7426901 
      Neg Pred Value            Precision               Recall 
           0.6918033            0.7426901            0.5746606 
                  F1           Prevalence       Detection Rate 
           0.6479592            0.4642857            0.2668067 
Detection Prevalence    Balanced Accuracy 
           0.3592437            0.7010558

In [42]:
roc_knn <- multiclass.roc(TestClasses, as.numeric(pred_knn))
roc_knn



Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.7172

In [43]:
roc_obj <- roc(TestClasses, as.numeric(pred_knn))
plot(roc_obj, main="Curva ROC")

Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [44]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_knn))

Data: as.numeric(pred_knn) in 171 controls (TestClasses fast) < 305 cases (TestClasses NotFast).
Area under the curve: 0.7172

Probabilità per ogni classe

In [45]:
pred_knn.prob

fast,NotFast
0.946248896,0.05375110
0.354236654,0.64576335
0.939045526,0.06095447
0.671031900,0.32896810
0.644553212,0.35544679
0.810319795,0.18968020
0.833310263,0.16668974
0.619687008,0.38031299
0.046246144,0.95375386
0.382107747,0.61789225
